In [1]:
# import all required libraries
import pandas as pd
import numpy as np

In [2]:
# read data from csv
portfolio_df = pd.read_csv("data/portfolio.csv")
profile_df = pd.read_csv("data/profile.csv")
transcript_df = pd.read_csv("data/transcript.csv")

del portfolio_df["Unnamed: 0"]
del profile_df["Unnamed: 0"]
del transcript_df["Unnamed: 0"]

In [3]:
portfolio_df.head()

,reward,difficulty,duration,offer_type,id,web,email,mobile,social
0,10,10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd,0,1,1,1
1,10,10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0,1,1,1,1
2,0,0,4,informational,3f207df678b143eea3cee63160fa8bed,1,1,1,0
3,5,5,7,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9,1,1,1,0
4,5,20,10,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7,1,1,0,0


In [4]:
profile_df.head()

,gender,age,id,became_member_on,income
0,F,39,68be06ca386d4c31939f3a4f0e3dd783,2017-02-12,65404.991568
1,F,55,0610b486422d4921ae7d2bf64640c50b,2017-07-15,112000.000000
2,M,30,38fe809add3b4fcf9315a9694bb96ff5,2018-07-12,65404.991568
3,F,75,78afa995795e4d85b5d9ceeca43f5fef,2017-05-09,100000.000000
4,M,53,a03223e636434f42ac4c3df47e8bac43,2017-08-04,65404.991568


In [5]:
transcript_df.head()

,event_offer_completed,event_offer_received,event_offer_viewed,event_transaction,person,value,time,value_type
0,0,1,0,0,78afa995795e4d85b5d9ceeca43f5fef,9b98b8c7a33c4b65b9aebfe6a799e6d9,0,offer_id
1,0,1,0,0,a03223e636434f42ac4c3df47e8bac43,0b1e1539f2cc45b7b9fa7c272da2e1d7,0,offer_id
2,0,1,0,0,e2127556f4f64592b11af22de27a7932,2906b810c7d4411798c6938adc9daaa5,0,offer_id
3,0,1,0,0,8ec6ce2a7e7949b1bf142def7d0e0586,fafdcd668e3743c1bb461111dcafc2a4,0,offer_id
4,0,1,0,0,68617ca6246f4fbc85e91a2a49552598,4d5c57ea9a6940dd891ad53e9dbe8da0,0,offer_id


In [6]:
# assign numerical id for customers across all dataframes
user_id_map = {}
last_assigned_id = 0

def assign_user_id(user):
    global last_assigned_id
    if user not in user_id_map.keys():
        last_assigned_id += 1
        user_id_map[user] = last_assigned_id  
    return user_id_map[user]

profile_df["user_id"] = profile_df["id"].apply(assign_user_id)
profile_df.head()

,gender,age,id,became_member_on,income,user_id
0,F,39,68be06ca386d4c31939f3a4f0e3dd783,2017-02-12,65404.991568,1
1,F,55,0610b486422d4921ae7d2bf64640c50b,2017-07-15,112000.000000,2
2,M,30,38fe809add3b4fcf9315a9694bb96ff5,2018-07-12,65404.991568,3
3,F,75,78afa995795e4d85b5d9ceeca43f5fef,2017-05-09,100000.000000,4
4,M,53,a03223e636434f42ac4c3df47e8bac43,2017-08-04,65404.991568,5


In [7]:
transcript_df["user_id"] = transcript_df["person"].apply(assign_user_id)
transcript_df.head()

,event_offer_completed,event_offer_received,event_offer_viewed,event_transaction,person,value,time,value_type,user_id
0,0,1,0,0,78afa995795e4d85b5d9ceeca43f5fef,9b98b8c7a33c4b65b9aebfe6a799e6d9,0,offer_id,4
1,0,1,0,0,a03223e636434f42ac4c3df47e8bac43,0b1e1539f2cc45b7b9fa7c272da2e1d7,0,offer_id,5
2,0,1,0,0,e2127556f4f64592b11af22de27a7932,2906b810c7d4411798c6938adc9daaa5,0,offer_id,6
3,0,1,0,0,8ec6ce2a7e7949b1bf142def7d0e0586,fafdcd668e3743c1bb461111dcafc2a4,0,offer_id,7
4,0,1,0,0,68617ca6246f4fbc85e91a2a49552598,4d5c57ea9a6940dd891ad53e9dbe8da0,0,offer_id,8


In [8]:
# verify user id assigned is same across dataframes
def test_user_id(user_id):
    old_id = profile_df[profile_df["user_id"] == user_id]["id"].values[0]
    assigned_id = transcript_df[transcript_df["person"] == old_id]["user_id"].unique().tolist()
    assert len(assigned_id) == 1, "Expect only one unique user_id, got: {}".format(assigned_id)
    assert assigned_id[0] == user_id, "Not same user id! Expected: {} Got: {}".format(user_id, assigned_id[0])


# test for random user ids
test_user_id(1)

test_user_id(10)

test_user_id(16)

# delete old ids
del profile_df["id"]
del transcript_df["person"]

In [9]:
# assign numerical id for offers across all dataframes
offer_id_map = {}
last_assigned_id = 0

def assign_offer_id(offer):
    global last_assigned_id
    if offer not in offer_id_map.keys():
        last_assigned_id += 1
        offer_id_map[offer] = last_assigned_id  
    return offer_id_map[offer]

portfolio_df["offer_id"] = portfolio_df["id"].apply(assign_offer_id)
portfolio_df.head()

,reward,difficulty,duration,offer_type,id,web,email,mobile,social,offer_id
0,10,10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd,0,1,1,1,1
1,10,10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0,1,1,1,1,2
2,0,0,4,informational,3f207df678b143eea3cee63160fa8bed,1,1,1,0,3
3,5,5,7,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9,1,1,1,0,4
4,5,20,10,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7,1,1,0,0,5


In [10]:
# separate transactions from offer events
transaction_df = transcript_df[transcript_df["event_transaction"] == 1]
offer_df = transcript_df[transcript_df["event_transaction"] == 0]

offer_df["offer_id"] = offer_df["value"].apply(assign_offer_id)
offer_df.head()

/Users/kanusha/Desktop/Data Science/Starbucks Capstone/env/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,event_offer_completed,event_offer_received,event_offer_viewed,event_transaction,value,time,value_type,user_id,offer_id
0,0,1,0,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,0,offer_id,4,4
1,0,1,0,0,0b1e1539f2cc45b7b9fa7c272da2e1d7,0,offer_id,5,5
2,0,1,0,0,2906b810c7d4411798c6938adc9daaa5,0,offer_id,6,10
3,0,1,0,0,fafdcd668e3743c1bb461111dcafc2a4,0,offer_id,7,7
4,0,1,0,0,4d5c57ea9a6940dd891ad53e9dbe8da0,0,offer_id,8,2


In [11]:
# verify offer id assigned is same across dataframes
def test_offer_id(offer_id):
    old_id = portfolio_df[portfolio_df["offer_id"] == offer_id]["id"].values[0]
    assigned_id = offer_df[offer_df["value"] == old_id]["offer_id"].unique().tolist()
    assert len(assigned_id) == 1, "Expect only one unique user_id, got: {}".format(assigned_id)
    assert assigned_id[0] == offer_id, "Not same user id! Expected: {} Got: {}".format(user_id, assigned_id[0])


# test for random user ids
test_offer_id(1)
test_offer_id(4)
test_offer_id(7)

In [12]:
# combine back to transcript_df
offer_df["value"] = offer_df["offer_id"]
del offer_df["offer_id"]
del portfolio_df["id"]


# convert value for transactions into float
transaction_df["value"] = transaction_df["value"].astype(float)

transcript_df = pd.concat([offer_df, transaction_df], axis=0)
transcript_df.head()

/Users/kanusha/Desktop/Data Science/Starbucks Capstone/env/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/kanusha/Desktop/Data Science/Starbucks Capstone/env/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,event_offer_completed,event_offer_received,event_offer_viewed,event_transaction,value,time,value_type,user_id
0,0,1,0,0,4.0,0,offer_id,4
1,0,1,0,0,5.0,0,offer_id,5
2,0,1,0,0,10.0,0,offer_id,6
3,0,1,0,0,7.0,0,offer_id,7
4,0,1,0,0,2.0,0,offer_id,8


In [13]:
# convert value_type in transcript_df into one hot
transcript_df = pd.concat([pd.get_dummies(transcript_df["value_type"], prefix="value_type"), transcript_df.drop(["value_type"], axis=1)], axis=1)
transcript_df.head()

,value_type_amount,value_type_offer_id,event_offer_completed,event_offer_received,event_offer_viewed,event_transaction,value,time,user_id
0,0,1,0,1,0,0,4.0,0,4
1,0,1,0,1,0,0,5.0,0,5
2,0,1,0,1,0,0,10.0,0,6
3,0,1,0,1,0,0,7.0,0,7
4,0,1,0,1,0,0,2.0,0,8


In [14]:
# convert gender in profile_df to one hot
profile_df = pd.concat([pd.get_dummies(profile_df["gender"], prefix="gender"), profile_df.drop(["gender"], axis=1)], axis=1)
profile_df.head()

,gender_F,gender_M,gender_O,age,became_member_on,income,user_id
0,1,0,0,39,2017-02-12,65404.991568,1
1,1,0,0,55,2017-07-15,112000.000000,2
2,0,1,0,30,2018-07-12,65404.991568,3
3,1,0,0,75,2017-05-09,100000.000000,4
4,0,1,0,53,2017-08-04,65404.991568,5


In [15]:
# covert became_member_on to became_member_year in profile_df
def get_year(date):
    return date.year

profile_df["became_member_year"] = pd.to_datetime(profile_df["became_member_on"]).apply(get_year)
del profile_df["became_member_on"]

profile_df.head()

,gender_F,gender_M,gender_O,age,income,user_id,became_member_year
0,1,0,0,39,65404.991568,1,2017
1,1,0,0,55,112000.000000,2,2017
2,0,1,0,30,65404.991568,3,2018
3,1,0,0,75,100000.000000,4,2017
4,0,1,0,53,65404.991568,5,2017


In [16]:
# convert offer_type to one hot in portfolio_df
portfolio_df = pd.concat([pd.get_dummies(portfolio_df["offer_type"], prefix="offer_type"), portfolio_df.drop(["offer_type"], axis=1)], axis=1)
portfolio_df.head()

,offer_type_bogo,offer_type_discount,offer_type_informational,reward,difficulty,duration,web,email,mobile,social,offer_id
0,1,0,0,10,10,7,0,1,1,1,1
1,1,0,0,10,10,5,1,1,1,1,2
2,0,0,1,0,0,4,1,1,1,0,3
3,1,0,0,5,5,7,1,1,1,0,4
4,0,1,0,5,20,10,1,1,0,0,5


In [17]:
transaction_df.head()

,event_offer_completed,event_offer_received,event_offer_viewed,event_transaction,value,time,value_type,user_id
12654,0,0,0,1,0.83,0,amount,42
12657,0,0,0,1,34.56,0,amount,56
12659,0,0,0,1,13.23,0,amount,64
12670,0,0,0,1,19.51,0,amount,135
12671,0,0,0,1,18.97,0,amount,152


In [18]:
# add total amount spent by each user into profile_df
user_total_amt = {}

def add_user_expendiure(row):
    if row["user_id"] in user_total_amt.keys():
        user_total_amt[row["user_id"]] += row["value"]
    user_total_amt[row["user_id"]] = row["value"]

transaction_df.apply(add_user_expendiure, axis=1)

def get_user_expenditure(user_id):
    if user_id in user_total_amt.keys():
        return user_total_amt[user_id]
    return 0

profile_df["total_amt"] = profile_df["user_id"].apply(get_user_expenditure)
profile_df.head()

,gender_F,gender_M,gender_O,age,income,user_id,became_member_year,total_amt
0,1,0,0,39,65404.991568,1,2017,5.21
1,1,0,0,55,112000.000000,2,2017,23.22
2,0,1,0,30,65404.991568,3,2018,4.09
3,1,0,0,75,100000.000000,4,2017,26.56
4,0,1,0,53,65404.991568,5,2017,0.06


In [19]:
# our target variable for models will be the category of offer the user mostly responds to

# 0 - Users who don't complete any offers
# 1 - Users who have most completed offer type Bogo
# 2 - Users who have most completed offer type Discount
# 3 - they will complete the offer without receiving or viewing

# our model will learn the pattern in their demographics and transactions and predict the offer type for them.

model_df = profile_df

In [20]:
def get_event_transactions(user_transactions, event_type):
    return user_transactions[user_transactions["event_offer_{}".format(event_type)] == 1]
    
def get_num_offers(user_id, offer_id):
    user_transactions = transcript_df[transcript_df["user_id"] == user_id]
    ret_values = []
    for e_type in event_types:
        event_transactions = get_event_transactions(user_transactions, e_type)
        ret_values.append(event_transactions[event_transactions.value == offer_id].shape[0])
    return ret_values


event_types = ["received", "viewed", "completed"]

"""
for offer_id in portfolio_df["offer_id"].tolist():
    print("Running get_num_offers for offer", offer_id)
    ret = np.array(model_df["user_id"].apply(get_num_offers, args=(offer_id, )).tolist())
    for i, e_type in enumerate(event_types):
        print("Getting {} events for offer {}".format(e_type, offer_id))
        model_df["offer_{}_{}".format(offer_id, e_type)] = ret[:, i]
"""

'\nfor offer_id in portfolio_df["offer_id"].tolist():\n    print("Running get_num_offers for offer", offer_id)\n    ret = np.array(model_df["user_id"].apply(get_num_offers, args=(offer_id, )).tolist())\n    for i, e_type in enumerate(event_types):\n        print("Getting {} events for offer {}".format(e_type, offer_id))\n        model_df["offer_{}_{}".format(offer_id, e_type)] = ret[:, i]\n'

In [22]:
model_df = pd.read_csv("data/processed/model.csv")
del model_df["Unnamed: 0"]

model_df.head()

,gender_F,gender_M,gender_O,age,income,user_id,became_member_year,total_amt,offer_1_received,offer_1_viewed,...,offer_7_completed,offer_8_received,offer_8_viewed,offer_8_completed,offer_9_received,offer_9_viewed,offer_9_completed,offer_10_received,offer_10_viewed,offer_10_completed
0,1,0,0,39,65404.991568,1,2017,5.21,0,0,...,1,0,0,0,0,0,0,1,1,0
1,1,0,0,55,112000.000000,2,2017,23.22,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,1,0,30,65404.991568,3,2018,4.09,0,0,...,0,1,1,0,0,0,0,0,0,0
3,1,0,0,75,100000.000000,4,2017,26.56,1,1,...,0,1,1,0,1,1,1,0,0,0
4,0,1,0,53,65404.991568,5,2017,0.06,0,0,...,0,1,0,0,0,0,0,0,0,0


In [ ]:
model_df.to_csv("data/processed/model.csv")

In [ ]:
# save all dataframes to csv
portfolio_df.to_csv("data/processed/portfolio.csv")
profile_df.to_csv("data/processed/profile.csv")
transcript_df.to_csv("data/processed/transcript.csv")